In [18]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [19]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [37]:
import minsearch


In [38]:
import json
with open('documents.json', 'rt') as f_in:
    docs_raw=json.load(f_in)

In [39]:
docs_raw[1].keys()
docs_raw[1]['documents'][1].keys()

dict_keys(['text', 'section', 'question'])

In [40]:
#docs_raw is a list of 3 dictionaries
print(len(docs_raw))
# each dictionary has two keys
print(docs_raw[0].keys())
# course key has one item
print(docs_raw[2]['course'])
# documents key has a list dictionaries which add up to 435
print(len(docs_raw[2]['documents']))
# each of those dictionaries has three keys namely
print(docs_raw[0]['documents'][0].keys())

3
dict_keys(['course', 'documents'])
mlops-zoomcamp
138
dict_keys(['text', 'section', 'question'])


In [41]:
len(docs_raw)
#docs_raw[0]
#len(docs_row[0])

3

In [42]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [43]:
print(len(documents))
(documents[0].keys())

948


dict_keys(['text', 'section', 'question', 'course'])

In [44]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]  
)


In [45]:
q = 'the course has already started, can I still enroll?'

In [46]:
index.fit(documents)

In [47]:
boost ={'question':3.0, 'section':0.5}
results= index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5

    
)

In [48]:
#print(results)
#results


In [49]:
from openai import OpenAI
client = OpenAI()
#client = OpenAI()

In [50]:
! python -V

Python 3.10.14


In [51]:
import os
os.environ
envir=os.environ

#api_key = os.environ.get("OPENAI_API_KEY")


In [52]:
# if 'OPENAI_API_KEY' in envir:
#     print(envir['OPENAI_API_KEY'])
# else:
#     print(False)

In [53]:
response = client.chat.completions.create(
    #model='gpt-3.5-turbo',
    model='gpt-4',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"The capability to enroll in a course after it has started depends on the specific policies of the institution or organization running the course. In some cases, late enrollment is allowed, but in others it is not. It would be best to directly contact the course provider to find out their rules and whether late enrollment is a possibility for the course you're interested in."

In [54]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results
query=search(q)
query

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [55]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [56]:
prompt=build_prompt(query=q, search_results=results)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related 

In [57]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [58]:
llm(prompt)

'Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks even if you register late. However, be mindful of the deadlines for turning in the final projects to avoid last-minute stress.'

In [59]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [60]:
rag(query)

'To run Kafka, here are the instructions depending on the specific scenario:\n\n1. **Java Kafka Producer/Consumer/KStreams**:\n   - Navigate to your project directory.\n   - Execute:\n     ```bash\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n   Replace `<jar_name>` with the appropriate jar file name.\n\n2. **Python Kafka**:\n   - If you encounter any issues such as "Module \'kafka\' not found", create and use a virtual environment.\n   - Steps to create and use a virtual environment:\n\n     To create:\n     ```bash\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n\n     To activate the virtual environment:\n     ```bash\n     source env/bin/activate\n     ```\n\n     To deactivate:\n     ```bash\n     deactivate\n     ```\n\nFor **Windows**, activate the virtual environment with:\n```bash\nenv\\Scripts\\activate\n```\n\nMake sure any Docker images required are 

In [61]:
rag('the course has started. Can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You will be eligible to submit the homeworks, but make sure to pay attention to the deadlines for turning in the final projects.'

In [62]:
rag('Tell me about Kilimanjaro Mountain in Kenya')

'It appears that the context provided does not contain any information about Kilimanjaro Mountain in Kenya. Based on the context given, it primarily addresses technical issues related to analytics engineering, dbt, BigQuery, pyspark, and other similar topics.\n\nHowever, for your information, Kilimanjaro Mountain is actually located in Tanzania, not Kenya. It is the highest mountain in Africa, standing at 5,895 meters (19,341 feet) above sea level. It is a major climbing destination and consists of three volcanic cones: Kibo, Mawenzi, and Shira.\n\nIf you need more detailed information, I recommend consulting geographical references or travel guides that specialize in Tanzania and its landmarks.'

In [63]:
def aigenerate(prompt):
    answer = llm(prompt)
    return answer

In [64]:
aigenerate('Tell me about Kilimanjaro Mountain in Kenya')

'Mount Kilimanjaro is often mistakenly thought to be in Kenya, but it actually lies in northern Tanzania, near the border with Kenya. It is the highest peak in Africa, standing at approximately 5,895 meters (19,341 feet) above sea level. The mountain is a dormant stratovolcano and has three distinct cones: Kibo, Mawenzi, and Shira. Kibo is the highest summit and home to the famous Uhuru Peak.\n\nWhile the mountain itself is in Tanzania, it can be seen from Kenya, particularly from Amboseli National Park, which is a popular vantage point due to its excellent views of Kilimanjaro.\n\nKilimanjaro is a major destination for adventurous travelers and mountaineers from around the world. Climbing routes vary in difficulty and include the Marangu, Machame, Lemosho, Rongai, and Umbwe routes, each offering unique landscapes and experiences.\n\nThe mountain is also known for its diverse ecosystems, ranging from tropical rainforest at its base to alpine desert and arctic conditions at its summit. 

In [65]:
para=aigenerate('''please improve the following personal statement to include results and positive impact I had to attract employers "I am a passionate and dedicated Mathematics Teacher with a strong background in both teaching and management. With over ten years of experience in education, I have consistently demonstrated my ability to inspire and challenge students, fostering a love for mathematics and nurturing academic excellence. My professional journey, marked by my roles as a supply teacher, English coordinator, regional manager, and machine learning developer, has equipped me with a diverse skill set and a mature, responsible approach to any task or situation.

Throughout my career, I have honed my ability to deliver high-quality lessons tailored to the needs and abilities of students in Key Stage 3 and Key Stage 4. My tenure as a supply teacher for Zen Educate allowed me to effectively teach mathematical concepts and manage classroom activities, ensuring that students remained engaged and on task. This experience has strengthened my capability to create stimulating learning environments that encourage active participation and foster critical thinking skills.

As an English Coordinator and instructor at Laureate Vocational Saudi Arabia, I managed multiple departments and conducted formal lesson observations, ensuring the delivery of quality teaching and learning. My role involved developing teaching improvement plans, managing timetables, and providing comprehensive support to senior managers with timely information and data for operational decisions. This has enhanced my ability to collaborate effectively with colleagues and contribute to the design and implementation of a cohesive mathematics curriculum.

My experience as a regional manager and English teacher at QEHC further developed my leadership and communication skills. Managing three English departments across multiple sites required a high level of organization, emotional intelligence, and the ability to communicate effectively through written and oral means. I consistently ensured that policies and procedures were understood and implemented, driving learner outcomes and supporting the personal development of my team members.

I am committed to continuous professional development, as evidenced by my qualifications in observation and feedback, a 150-hour master TEFL course, and a Cambridge CELTA certification. My academic background, including an MEng ACGI (Hons) in Chemical Engineering from Imperial College London, has provided me with a solid foundation in mathematics and problem-solving, which I am eager to share with my students.

In addition to my teaching and management experience, I have a proven track record of achieving positive outcomes for students through innovative teaching methods and the use of diverse resources. My ability to engage with parents and guardians, provide constructive feedback, and address any concerns has been integral to my success as an educator.

I am excited about the opportunity to join Ghars International, where I can leverage my extensive experience and passion for mathematics to inspire curiosity and enthusiasm in students. I am confident in my ability to work collaboratively within a team, establish positive relationships, and contribute to the wider life of the school community. I look forward to the possibility of bringing my expertise and dedication to your esteemed institution."''')

In [66]:
para.strip('/n')

"I am an enthusiastic and devoted Mathematics Teacher with over ten years of experience, underpinned by a robust combination of teaching and management skills. My career in education has been marked by my dedication to inspiring and challenging students, fostering a deep love for mathematics, and nurturing academic excellence. The array of roles I have undertaken—ranging from a supply teacher and English coordinator to a regional manager and machine learning developer—reflects my diverse skill set and responsible approach to any task or situation.\n\nThroughout my career, I have consistently delivered high-quality lessons tailored to the needs and abilities of students in Key Stages 3 and 4. During my tenure as a supply teacher with Zen Educate, I effectively taught mathematical concepts and managed classroom activities, ensuring high engagement levels and on-task behavior. This resulted in a marked improvement in students' comprehension and performance, showcasing my ability to cultiv

In [67]:
para.replace('\n','')

"I am an enthusiastic and devoted Mathematics Teacher with over ten years of experience, underpinned by a robust combination of teaching and management skills. My career in education has been marked by my dedication to inspiring and challenging students, fostering a deep love for mathematics, and nurturing academic excellence. The array of roles I have undertaken—ranging from a supply teacher and English coordinator to a regional manager and machine learning developer—reflects my diverse skill set and responsible approach to any task or situation.Throughout my career, I have consistently delivered high-quality lessons tailored to the needs and abilities of students in Key Stages 3 and 4. During my tenure as a supply teacher with Zen Educate, I effectively taught mathematical concepts and managed classroom activities, ensuring high engagement levels and on-task behavior. This resulted in a marked improvement in students' comprehension and performance, showcasing my ability to cultivate 

# elasticsearch

In [68]:
from elasticsearch import Elasticsearch

In [69]:
es_client=Elasticsearch('http://localhost:9200')

In [70]:
es_client.info()

ObjectApiResponse({'name': 'fa922577a214', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0SrJ1D-zTeqZyr_L0VZR2Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [71]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name='course-questions'
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/bRKgAPSETDqwNIZcPX7mBw] already exists')

In [72]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [73]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:26<00:00, 36.17it/s]


In [74]:
query='I just discovered the course. Can I still join it?'

In [75]:
def elastic_search(query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response=es_client.search(index=index_name, body=search_query)
    result_doc=[]
    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])
    return result_doc

In [76]:
elastic_search(query)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'T

In [77]:
query = 'how do I run kafka?'

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [78]:
query='I just discovered the course. Can I still join it?'
rag(query)

'Yes, you can still join the course even if it has already started. You won’t be able to submit some of the homeworks, but you can still participate in the course activities. If you are looking to get a certificate, you need to submit 2 out of the 3 course projects and review 3 of your peers’ projects by the given deadline. This means that even if you join the course late, as long as you complete these requirements, you will still be eligible for a certificate.'

In [87]:
query='Which course do you have?'
rag(query)

'We have a course on Data Engineering.'

In [86]:
query='When is the machine learning course starting? and how long is it running for?'
rag(query)

'The machine learning course runs for approximately 4 months. The exact start date is not provided in the context. If you want to do extra activities such as an extra project or writing an article, it may take more time.'

In [81]:
query='When is the machine learning zoomcamp starting? and how long is it running for?'
rag(query)

'The provided context does not contain information about the starting date or the duration of the Machine Learning Zoomcamp. To find that information, you might need to contact the organizers directly or refer to the official website or course materials for the Zoomcamp.'

In [82]:
query='Will there be a certificate given to those who finish the course successfully?'
rag(query)

'The provided context does not include any information regarding the issuance of certificates for those who finish the course successfully. Please refer to other sections of the FAQ database or contact the course coordinator for more details.'

In [85]:
query='how many courses do you have?'
rag(query)

'The CONTEXT provided does not contain information about the number of courses. Therefore, I cannot provide an answer to the QUESTION based on the available CONTEXT.'